## Compile Taxi Trips Aggregated Table

In [47]:
# Compile taxi trips aggregated table ftom
# - taxi trips data
# - community areas data 
# - weather data 

In [48]:
# Import StringIO class in the io module that allows to treat a string as if
# it were a file object
from io import StringIO
import os

# Import load_dotenv function from the python-dotenv library
from dotenv import load_dotenv

# Import AWS SDK for Python to interact with AWS services from python code
import boto3

# Import Pandas data manipulation and analysis library
import pandas as pd


In [49]:
# Looks for a .env file (by default, in the current directory), reads it, and
# adds the variables to the environment.
load_dotenv()


True

In [50]:
# Retrieving sensitive credentials (an AWS Access Key and Secret Key) from
# environment variables using the os module.
aws_access_key_id = os.getenv("AWS_ACCESS_KEY")
aws_secret_key = os.getenv("AWS_SECRET_KEY")


In [51]:
def read_csv_from_s3(bucket: str, path: str, filename: str) -> pd.DataFrame:
    """
    Download cscv file from S3 bucket.

    Parameters:
    bucket (str): The bucket where the files at.
    path (str): The folder to the file.
    filename (str): Original file name.

    Returns:
        pd:DataFrame - the dataframe of the downloaded file
    """
    s3 = boto3.client(
        "s3", aws_access_key_id=aws_access_key_id, aws_secret_access_key=aws_secret_key
    )
    full_path = f"{path}{filename}"
    object = s3.get_object(Bucket=bucket, Key=full_path)
    object = object["Body"].read().decode("utf-8")
    output_df = pd.read_csv(StringIO(object))
    return output_df


In [52]:
# Use boto3 to connect to AWS S3, then reads several CSV files from specific
# folders (or "paths") in an S3 bucket, loading each one into memory using a
# custom function called read_csv_from_s3
s3 = boto3.client(
    "s3", aws_access_key_id=aws_access_key_id, aws_secret_access_key=aws_secret_key
)
bucket = "cubix-chicago-taxi-ke"

community_areas_path = "transformed_data/community_areas/"
company_path = "transformed_data/company/"
date_path = "transformed_data/date/"
payment_type_path = "transformed_data/payment_type/"

# Note: Taxi and wetaher data files processed separately!
taxi_trips_path = "transformed_data/taxi_trips/"
weather_path = "transformed_data/weather/"

community_areas = read_csv_from_s3(
    bucket=bucket, path=community_areas_path, filename="community_areas_master.csv"
)
company = read_csv_from_s3(
    bucket=bucket, path=company_path, filename="company_master.csv"
)
date = read_csv_from_s3(bucket=bucket, path=date_path, filename="weather_data_date.csv")
payment_type = read_csv_from_s3(
    bucket=bucket, path=payment_type_path, filename="payment_type_master.csv"
)


In [53]:
# Process taxi data files
trips_list = []
for file in s3.list_objects(Bucket=bucket, Prefix=taxi_trips_path).get("Contents", []):
    taxi_trip_key = file["Key"]
    filename = taxi_trip_key.split("/")[-1]

    if filename and filename.endswith(".csv"):
        filename = taxi_trip_key.split("/")[-1]

        # print(filename)
        trip = read_csv_from_s3(bucket, taxi_trips_path, filename)
        trips_list.append(trip)
        print(f"{filename} has been added.")
trips = pd.concat(trips_list, ignore_index=True)
trips.head()


taxi_2025-01-22.csv has been added.
taxi_2025-01-23.csv has been added.
taxi_2025-01-24.csv has been added.
taxi_2025-01-25.csv has been added.
taxi_2025-01-26.csv has been added.
taxi_2025-01-27.csv has been added.
taxi_2025-01-28.csv has been added.
taxi_2025-01-29.csv has been added.
taxi_2025-01-30.csv has been added.
taxi_2025-01-31.csv has been added.
taxi_2025-02-01.csv has been added.
taxi_2025-02-02.csv has been added.
taxi_2025-02-03.csv has been added.
taxi_2025-02-04.csv has been added.
taxi_2025-02-05.csv has been added.
taxi_2025-02-06.csv has been added.
taxi_2025-02-07.csv has been added.
taxi_2025-02-08.csv has been added.
taxi_2025-02-09.csv has been added.
taxi_2025-02-10.csv has been added.
taxi_2025-02-11.csv has been added.
taxi_2025-02-12.csv has been added.
taxi_2025-02-13.csv has been added.
taxi_2025-02-14.csv has been added.
taxi_2025-02-15.csv has been added.
taxi_2025-02-16.csv has been added.
taxi_2025-02-17.csv has been added.
taxi_2025-02-18.csv has been

,trip_id,taxi_id,trip_start_timestamp,trip_end_timestamp,trip_seconds,trip_miles,pickup_community_area_id,dropoff_community_area_id,fare,tips,tolls,extras,trip_total,pickup_centroid_latitude,pickup_centroid_longitude,dropoff_centroid_latitude,dropoff_centroid_longitude,datetime_for_weather,payment_type_id,company_id
0,a407ceae21e3ee9e17ec7b2162405eb52cfa25ed,bff345061f21d319fb6ec44e0ab2bb05478383a21e1ecb...,2025-01-22T23:45:00.000,2025-01-22T23:45:00.000,510,1.55,8,8,8.00,0.0,0.0,0.00,8.00,41.899602,-87.633308,41.899602,-87.633308,2025-01-22 23:00:00,3,4
1,fece1b60d6746287bfd227e424114d0f1a46c4e2,78d45b51ff03d70f0ba2afda334acf93f678a179aaaaca...,2025-01-22T23:45:00.000,2025-01-22T23:45:00.000,166,0.00,8,8,5.00,0.0,0.0,0.00,5.00,41.899602,-87.633308,41.899602,-87.633308,2025-01-22 23:00:00,3,12
2,fadb4ea6d6a20a0ec21be5db3faa677ce428caae,3618045f9110d4d88482266ade23659c1a50d32ac37f20...,2025-01-22T23:45:00.000,2025-01-22T23:45:00.000,0,0.00,24,24,3.25,0.0,0.0,11.25,14.50,41.901207,-87.676356,41.901207,-87.676356,2025-01-22 23:00:00,2,2
3,f92db77c2a17b2d14aae584f8273a5d687fe5d18,8ff96b8befe47605908893fbb12cc9a4845f2b6f9bfc38...,2025-01-22T23:45:00.000,2025-01-22T23:45:00.000,660,5.10,8,6,15.25,4.0,0.0,1.00,20.25,41.899602,-87.633308,41.944227,-87.655998,2025-01-22 23:00:00,2,2
4,f26504c75b0f4c29cfc26a06614c66b61b66be34,1a248eb87ae578f15af2a14c2eb7b4b264f6babab9ad9c...,2025-01-22T23:45:00.000,2025-01-23T00:00:00.000,812,4.85,8,6,15.00,0.0,0.0,0.00,15.00,41.899602,-87.633308,41.944227,-87.655998,2025-01-22 23:00:00,3,1


In [54]:
# Process weather data files
weather_list = []
for file in s3.list_objects(Bucket=bucket, Prefix=weather_path).get("Contents", []):
    weather_key = file["Key"]
    filename = weather_key.split("/")[-1]

    if filename and filename.endswith(".csv"):
        filename = weather_key.split("/")[-1]
        # print(filename)
        weather_daily = read_csv_from_s3(bucket, weather_path, filename)
        weather_list.append(weather_daily)
        print(f"{filename} has been added.")
weather = pd.concat(weather_list, ignore_index=True)
weather.head()


weather_2025-01-22.csv has been added.
weather_2025-01-23.csv has been added.
weather_2025-01-24.csv has been added.
weather_2025-01-25.csv has been added.
weather_2025-01-26.csv has been added.
weather_2025-01-27.csv has been added.
weather_2025-01-28.csv has been added.
weather_2025-01-29.csv has been added.
weather_2025-01-30.csv has been added.
weather_2025-01-31.csv has been added.
weather_2025-02-01.csv has been added.
weather_2025-02-02.csv has been added.
weather_2025-02-03.csv has been added.
weather_2025-02-04.csv has been added.
weather_2025-02-05.csv has been added.
weather_2025-02-06.csv has been added.
weather_2025-02-07.csv has been added.
weather_2025-02-08.csv has been added.
weather_2025-02-09.csv has been added.
weather_2025-02-10.csv has been added.
weather_2025-02-11.csv has been added.
weather_2025-02-12.csv has been added.
weather_2025-02-13.csv has been added.
weather_2025-02-14.csv has been added.
weather_2025-02-15.csv has been added.
weather_2025-02-16.csv ha

,datetime,tempretaure,wind_speed,rain,precipitation
0,2025-01-22 00:00:00,-17.8,12.6,0.0,0.0
1,2025-01-22 01:00:00,-17.9,13.4,0.0,0.0
2,2025-01-22 02:00:00,-17.9,14.0,0.0,0.0
3,2025-01-22 03:00:00,-18.0,15.5,0.0,0.0
4,2025-01-22 04:00:00,-18.0,16.4,0.0,0.0


In [55]:
# Join two DataFrames (trips and weather) based on matching values
trips_full = pd.merge(
    trips, weather, left_on="datetime_for_weather", right_on="datetime", how="inner"
)
trips_full = trips_full.drop(columns=["datetime"])


In [56]:
# Join two DataFrames (trips_full and company) based on matching values
trips_full = pd.merge(
    trips_full, company, left_on="company_id", right_on="company_id", how="inner"
)
trips_full = trips_full.drop(columns=["company_id"])


In [57]:
# Join two DataFrames (trips_full and payment_type) based on matching values
trips_full = pd.merge(
    trips_full,
    payment_type,
    left_on="payment_type_id",
    right_on="payment_type_id",
    how="inner",
)
trips_full = trips_full.drop(columns=["payment_type_id"])


In [58]:
# Join two DataFrames (trips_full and community_areas) based on
# "pickup_community_area_id" matching values
trips_full = pd.merge(
    trips_full,
    community_areas,
    left_on="pickup_community_area_id",
    right_on="area_code",
    how="inner",
)
trips_full = trips_full.drop(columns=["pickup_community_area_id"])
trips_full.rename(
    columns={"community_name": "pickup_community_area_name"}, inplace=True
)


In [59]:
# Join two DataFrames (trips_full and community_areas) based on
# "dropoff_community_area_id" matching values
trips_full = pd.merge(
    trips_full,
    community_areas,
    left_on="dropoff_community_area_id",
    right_on="area_code",
    how="inner",
)
trips_full = trips_full.drop(columns=["dropoff_community_area_id"])
trips_full.rename(
    columns={"community_name": "dropoff_community_area_name"}, inplace=True
)


In [60]:
# Controlling DataFrame display so you can actually see your full data without
# stuff getting cut off
pd.set_option("display.max_columns", 50)
trips_full.head()


,trip_id,taxi_id,trip_start_timestamp,trip_end_timestamp,trip_seconds,trip_miles,fare,tips,tolls,extras,trip_total,pickup_centroid_latitude,pickup_centroid_longitude,dropoff_centroid_latitude,dropoff_centroid_longitude,datetime_for_weather,tempretaure,wind_speed,rain,precipitation,company,payment_type,area_code_x,pickup_community_area_name,area_code_y,dropoff_community_area_name
0,a407ceae21e3ee9e17ec7b2162405eb52cfa25ed,bff345061f21d319fb6ec44e0ab2bb05478383a21e1ecb...,2025-01-22T23:45:00.000,2025-01-22T23:45:00.000,510,1.55,8.00,0.0,0.0,0.00,8.00,41.899602,-87.633308,41.899602,-87.633308,2025-01-22 23:00:00,-5.1,15.9,0.0,0.0,Flash Cab,Cash,8,Near North Side,8,Near North Side
1,fece1b60d6746287bfd227e424114d0f1a46c4e2,78d45b51ff03d70f0ba2afda334acf93f678a179aaaaca...,2025-01-22T23:45:00.000,2025-01-22T23:45:00.000,166,0.00,5.00,0.0,0.0,0.00,5.00,41.899602,-87.633308,41.899602,-87.633308,2025-01-22 23:00:00,-5.1,15.9,0.0,0.0,Tac - Yellow Cab Association,Cash,8,Near North Side,8,Near North Side
2,fadb4ea6d6a20a0ec21be5db3faa677ce428caae,3618045f9110d4d88482266ade23659c1a50d32ac37f20...,2025-01-22T23:45:00.000,2025-01-22T23:45:00.000,0,0.00,3.25,0.0,0.0,11.25,14.50,41.901207,-87.676356,41.901207,-87.676356,2025-01-22 23:00:00,-5.1,15.9,0.0,0.0,Taxi Affiliation Services,Credit Card,24,West Town,24,West Town
3,f92db77c2a17b2d14aae584f8273a5d687fe5d18,8ff96b8befe47605908893fbb12cc9a4845f2b6f9bfc38...,2025-01-22T23:45:00.000,2025-01-22T23:45:00.000,660,5.10,15.25,4.0,0.0,1.00,20.25,41.899602,-87.633308,41.944227,-87.655998,2025-01-22 23:00:00,-5.1,15.9,0.0,0.0,Taxi Affiliation Services,Credit Card,8,Near North Side,6,Lake View
4,f26504c75b0f4c29cfc26a06614c66b61b66be34,1a248eb87ae578f15af2a14c2eb7b4b264f6babab9ad9c...,2025-01-22T23:45:00.000,2025-01-23T00:00:00.000,812,4.85,15.00,0.0,0.0,0.00,15.00,41.899602,-87.633308,41.944227,-87.655998,2025-01-22 23:00:00,-5.1,15.9,0.0,0.0,Sun Taxi,Cash,8,Near North Side,6,Lake View


In [61]:
# Display date
date


,datetime,tempretaure,wind_speed,rain,precipitation
0,2025-01-25 00:00:00,-8.9,14.4,0.0,0.0
1,2025-01-25 01:00:00,-7.7,14.9,0.0,0.0
2,2025-01-25 02:00:00,-7.5,15.7,0.0,0.0
3,2025-01-25 03:00:00,-7.1,17.3,0.0,0.0
4,2025-01-25 04:00:00,-7.0,17.8,0.0,0.0
5,2025-01-25 05:00:00,-6.6,17.8,0.0,0.0
6,2025-01-25 06:00:00,-6.4,17.7,0.0,0.0
7,2025-01-25 07:00:00,-6.6,17.7,0.0,0.0
8,2025-01-25 08:00:00,-6.4,16.9,0.0,0.0
9,2025-01-25 09:00:00,-6.2,14.8,0.0,0.0


In [62]:
# Convert the datetime column in the date DataFrame to a proper datetime object
date["date"] = pd.to_datetime(date["datetime"])
# Convert the trip_start_timestamp column into a datetime.
trips_full["trip_start_timestamp"] = pd.to_datetime(trips_full["trip_start_timestamp"])
# Extract just the date part from the full timestamp, dropping the time
trips_full["trip_start_date"] = trips_full["trip_start_timestamp"].dt.date
trips_full["trip_start_date"]


0         2025-01-22
1         2025-01-22
2         2025-01-22
3         2025-01-22
4         2025-01-22
             ...    
446653    2025-02-22
446654    2025-02-22
446655    2025-02-22
446656    2025-02-22
446657    2025-02-22
Name: trip_start_date, Length: 446658, dtype: object

In [63]:
# Join two DataFrames (trips_full and date) based on matching values
trips_full = pd.merge(
    trips_full, date, left_on="trip_start_timestamp", right_on="date", how="inner"
)
trips_full = trips_full.drop(columns=["date"])
trips_full.head()


,trip_id,taxi_id,trip_start_timestamp,trip_end_timestamp,trip_seconds,trip_miles,fare,tips,tolls,extras,trip_total,pickup_centroid_latitude,pickup_centroid_longitude,dropoff_centroid_latitude,dropoff_centroid_longitude,datetime_for_weather,tempretaure_x,wind_speed_x,rain_x,precipitation_x,company,payment_type,area_code_x,pickup_community_area_name,area_code_y,dropoff_community_area_name,trip_start_date,datetime,tempretaure_y,wind_speed_y,rain_y,precipitation_y
0,4796fd2c33c70d9e4e1a19ba384ef34f6a1aa85b,a5bcdc46762283d695a098e0d5bc202cd0228bab131a5e...,2025-01-25 23:00:00,2025-01-25T23:30:00.000,1175,14.35,35.75,8.25,0.0,5.0,49.50,41.980264,-87.913625,41.922686,-87.649489,2025-01-25 23:00:00,1.1,20.9,0.0,0.0,5 Star Taxi,Credit Card,76,O'Hare,7,Lincoln Park,2025-01-25,2025-01-25 23:00:00,1.1,20.9,0.0,0.0
1,f99b704fcfdd7d57815ce6a2e078ad5335c1f720,5de9514abc0c5b1996ab022380ac899452405559793822...,2025-01-25 23:00:00,2025-01-25T23:30:00.000,1686,15.68,40.25,8.95,0.0,4.0,53.70,41.980264,-87.913625,41.944227,-87.655998,2025-01-25 23:00:00,1.1,20.9,0.0,0.0,City Service,Credit Card,76,O'Hare,6,Lake View,2025-01-25,2025-01-25 23:00:00,1.1,20.9,0.0,0.0
2,f89d43ca59a2b2996de0df4db977c21e72b3ede8,5ffab897d6aec7c42dbe4fb2ce017ad47ea52ded99ebbf...,2025-01-25 23:00:00,2025-01-25T23:15:00.000,612,1.79,8.62,0.00,0.0,0.0,9.12,41.899602,-87.633308,41.922686,-87.649489,2025-01-25 23:00:00,1.1,20.9,0.0,0.0,Top Cab,Mobile,8,Near North Side,7,Lincoln Park,2025-01-25,2025-01-25 23:00:00,1.1,20.9,0.0,0.0
3,f372cf89bd13e633fdab14dd434964f40416cae7,8b8b185d8a60ac5edd94c0494d0b9b9c9c86d54c0ab3dd...,2025-01-25 23:00:00,2025-01-25T23:30:00.000,1277,14.55,36.75,4.50,0.0,4.0,45.75,41.980264,-87.913625,41.922686,-87.649489,2025-01-25 23:00:00,1.1,20.9,0.0,0.0,City Service,Credit Card,76,O'Hare,7,Lincoln Park,2025-01-25,2025-01-25 23:00:00,1.1,20.9,0.0,0.0
4,edc8e40f5fbc6a126face429f5541c45844ec9ce,dff8b2296e1abd3c3470a2f0765b3f385f9a53c8df1cc3...,2025-01-25 23:00:00,2025-01-25T23:15:00.000,881,8.76,22.55,6.07,0.0,0.0,29.12,41.874005,-87.663518,41.794090,-87.592311,2025-01-25 23:00:00,1.1,20.9,0.0,0.0,Chicago Independents,Mobile,28,Near West Side,41,Hyde Park,2025-01-25,2025-01-25 23:00:00,1.1,20.9,0.0,0.0


In [64]:
# Save trips_full to a CSV file
trips_full.to_csv("trips_full.csv", index=False)
